In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Mesothelioma/GSE107754'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A novel genomic signature predicting FDG uptake in diverse metastatic tumors"
!Series_summary	"Purpose: Building a universal genomic signature predicting the intensity of FDG uptake in diverse metastatic tumors may allow us to understand better the biological processes underlying this phenomenon and their requirements of glucose uptake."
!Series_summary	"Methods: A balanced training set (n=71) of metastatic tumors including some of the most frequent histologies, with matched PET/CT quantification measurements and whole human genome gene expression microarrays, was used to build the signature. Selection of microarray features was carried out exclusively on the basis of their strong association with FDG uptake (as measured by SUVmean35) by means of univariate linear regression. A thorough bioinformatics study of these genes was performed and multivariable models were built by fitting several state of the art regression techniques to the training set

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data.
is_gene_available = True  # Based on the presence of microarray data.

# Analyze sample characteristics dictionary to detect availability and define rows for variables
sample_characteristics = {
    0: ['gender: Male', 'gender: Female'],
    1: ['dataset: Validation set', 'dataset: Training set'],
    2: [
        'biopsy location: Lung', 'biopsy location: Lymph node', 'biopsy location: Primary', 'biopsy location: Liver', 
        'biopsy location: Retroperitoneal implant', 'tissue: Pancreatic cancer', 'tissue: Esophagus cancer', 'tissue: Breast cancer', 
        'tissue: Colorectal cancer', 'tissue: Ovarian cancer', 'tissue: Head&neck cancer', 'tissue: Lung cancer', 'tissue: Malignant Melanoma', 
        'tissue: Endometrial cancer', 'tissue: Cervix cancer', 'tissue: Soft tissue sarcoma', 'tissue: Gastric cancer', 'tissue: Unknown primary', 
        'tissue: Malignant Mesothelioma', 'tissue: Thyroid cancer', 'tissue: Testes cancer', 'tissue: Non Hodgkin lymphoma', 'tissue: Merkel cell carcinoma', 
        'tissue: Vaginal cancer', 'tissue: Kidney cancer', 'tissue: Cervical cancer', 'tissue: Bile duct cancer', 'tissue: Urothelial cancer'
    ],
    3: [
        'suvmean35: 4.09', 'suvmean35: 8.36', 'suvmean35: 5.18', 'suvmean35: 10.74', 'suvmean35: 8.62', 'suvmean35: 8.02', 'suvmean35: 6.87', 
        'suvmean35: 4.93', 'suvmean35: 1.96', 'suvmean35: 8.83', 'suvmean35: 3.96', 'suvmean35: 3.38', 'suvmean35: 9.95', 'suvmean35: 5.19', 
        'suvmean35: 7.22', 'suvmean35: 5.02', 'suvmean35: 4.92', 'suvmean35: 4.99', 'suvmean35: 4.01', 'suvmean35: 2.52', 'suvmean35: 5.52', 
        'suvmean35: 8.38', 'suvmean35: 3.46', 'suvmean35: 4.07', 'suvmean35: 4.67', 'suvmean35: 7.09', 'suvmean35: 4.83', 'suvmean35: 6.7', 
        'suvmean35: 3.95', 'suvmean35: 5.03'
    ]
}

# Determine the data keys
trait_row = 2 if 'tissue: Malignant Mesothelioma' in sample_characteristics[2] else None
age_row = None  # No evidence of 'age' in sample characteristics
gender_row = 0 if 'gender: Male' in sample_characteristics[0] else None

# Define conversion functions
def convert_trait(value):
    val = value.split(':')[1].strip()
    return 1 if val == 'Malignant Mesothelioma' else 0

def convert_age(value):
    return None  # Placeholder if age data becomes available

def convert_gender(value):
    gender = value.split(':')[1].strip()
    return 1 if gender == 'Male' else 0 if gender == 'Female' else None

# Save cohort information
save_cohort_info('GSE107754', './preprocessed/Mesothelioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Mesothelioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Mesothelioma/trait_data/GSE107754.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2878070': [0, 1], 'GSM2878071': [0, 0], 'GSM2878072': [0, 1], 'GSM2878073': [0, 1], 'GSM2878074': [0, 0], 'GSM2878075': [0, 1], 'GSM2878076': [0, 0], 'GSM2878077': [0, 0], 'GSM2878078': [0, 0], 'GSM2878079': [0, 0], 'GSM2878080': [0, 0], 'GSM2878081': [0, 0], 'GSM2878082': [0, 0], 'GSM2891194': [0, 1], 'GSM2891195': [0, 1], 'GSM2891196': [0, 0], 'GSM2891197': [0, 0], 'GSM2891198': [0, 0], 'GSM2891199': [0, 0], 'GSM2891200': [0, 0], 'GSM2891201': [0, 1], 'GSM2891202': [0, 1], 'GSM2891203': [0, 0], 'GSM2891204': [0, 0], 'GSM2891205': [0, 1], 'GSM2891206': [0, 0], 'GSM2891207': [0, 0], 'GSM2891208': [0, 1], 'GSM2891209': [0, 1], 'GSM2891210': [0, 0], 'GSM2891211': [0, 0], 'GSM2891212': [0, 0], 'GSM2891213': [0, 0], 'GSM2891214': [0, 0], 'GSM2891215': [0, 1], 'GSM2891216': [0, 0], 'GSM2891217': [1, 1], 'GSM2891218': [0, 0], 'GSM2891219': [0, 0], 'GSM2891220': [0, 1], 'GSM2891221': [0, 0], 'GSM2891222': [0, 1], 'GSM2891223': [0, 0], 'GSM2891224': [0, 1], 'GSM2891225': [0, 1], 'GSM28912

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols in the gene annotation dictionary.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene_data dataframe.
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(preview_df(gene_data))


{'GSM2878070': [7.898410275, 8.337625324, 8.138943783, 7.597095505, 9.231020193], 'GSM2878071': [7.759717598, 9.2945473845, 9.361229596, 7.626464485, 9.510203889], 'GSM2878072': [7.602316177, 8.832979529, 8.513994134, 8.3897227655, 9.701203162], 'GSM2878073': [7.682897106, 10.130379023, 9.269931543, 9.743948959499999, 8.614287457], 'GSM2878074': [8.091468789, 8.9426286395, 9.537095395, 7.5950630785, 8.34944317], 'GSM2878075': [7.713223958, 11.734510947, 9.026285729, 9.908209515500001, 8.303010631], 'GSM2878076': [7.571070794, 8.6185850975, 8.261323704, 7.5620183585, 9.095688771], 'GSM2878077': [7.848485557, 10.7772292755, 9.41995522, 9.3455830075, 8.98381225], 'GSM2878078': [7.763564775, 9.2138486425, 8.954171939, 7.5645898525, 9.370094803], 'GSM2878079': [7.807678819, 9.72061828, 8.822408214, 7.754951997999999, 9.158876289], 'GSM2878080': [7.632001715, 8.205169727, 7.874668977, 7.640650282, 10.07776215], 'GSM2878081': [7.916931252, 9.734191202, 8.911633759, 7.783495661, 9.084567672], 

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Mesothelioma/gene_data/GSE107754.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, merged_data = judge_and_remove_biased_features(merged_data, 'Mesothelioma')

# 4. Save the cohort information.
save_cohort_info('GSE107754', './preprocessed/Mesothelioma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Mesothelioma/GSE107754.csv'
    merged_data.to_csv(csv_path)


For the feature 'Mesothelioma', the least common label is '1.0' with 1 occurrences. This represents 1.19% of the dataset.
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 35 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

